In [3]:
import logging
import os, sys

import gym
import gym_ple

pygame 1.9.4
Hello from the pygame community. https://www.pygame.org/contribute.html
couldn't import doomish
Couldn't import doom


In [14]:
# Create a breakout environment
env = gym.make('AchtungDieKurve-v0')
# Reset it, returns the starting frame
frame = env.reset()
# Render
env.render()
is_done = False
while not is_done:
  # Perform a random action, returns the new frame, reward and whether the game is over
  frame, reward, is_done, _ = env.step(env.action_space.sample())
  # Render
  env.render()
env.close()